In [1]:
import pymongo 
from pymongo import MongoClient
from netCDF4 import Dataset 
import collections
import bson

In [2]:
rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
sat_name="/home/awseed/data/SAT/crr-ph/2021/20210129/20210129030000-P1S-ABOM_CRRPH-PRJ_AEA132_2000-HIMAWARI8-AHI.nc" 

In [3]:
# open the data base and get the metadata 
rf3 = Dataset(rf3_name)
stn_id = rf3.getncattr("station_id")

In [4]:
# get the projection as a dictionary 
proj_attr = {}

proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
standard_parallel = rf3.variables['proj'].getncattr("standard_parallel")
proj_attr['standard_parallel'] =  standard_parallel.tolist()
proj_attr['longitude_of_central_meridian'] = rf3.variables['proj'].getncattr("longitude_of_central_meridian")
proj_attr['latitude_of_projection_origin'] = rf3.variables['proj'].getncattr("latitude_of_projection_origin")
proj_attr['false_easting'] = rf3.variables['proj'].getncattr("false_easting") 
proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
towgs84 = rf3.variables['proj'].getncattr("towgs84")
proj_attr['towgs84'] = towgs84.tolist()

print(proj_attr)

{'grid_mapping_name': 'albers_conical_equal_area', 'standard_parallel': [-18.0, -36.0], 'longitude_of_central_meridian': 132.0, 'latitude_of_projection_origin': 0.0, 'false_easting': 0.0, 'towgs84': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


In [ ]:
# setup the database, and a collection for the rf3 prc_crate data 
client = MongoClient()
rain_base = client["rain_base_fs"]
projection_id = f"stn_id_{stn_id}"
print(projection_id)
pjn_collection = pjn_db[projection_id]